# GenomeMLModels

## Introduction
This project aims to generate usable machine learning models for clinical datasets. 

The project incorporates the Hail data handling layer for inputting VCF/gVCF files, facilitating the analysis of datasets from various generations.

## Objectives
- To train a model on the Illumina Trusight Cancer(TM) and Trusight(TM) Hereditary Cancer panel VCFs.
- To annotate variants in new VCFs for potential pathogenicity based on the input phenotype.
- To select a decision boundary for single high probability variants, enabling quick downselection of variants or tagging of VCF files for potential monogenic variants.

This serves as a proof-of-concept for applying machine learning to genomic datasets using specified libraries.

## Data Structure
The project expects VCFs or gVCFs with genotype caller allele data and VEP annotations such as IMPACT, MAX_AF, and HGNC_ID. 
These annotations are converted into features for the model.

## Model Specification

A boosted decision tree model forms the backbone of our analytical approach, accommodating the complex nature of genomic data, which encompasses both categorical (strings) and continuous (float values) variables. This model will leverage existing open-source machine learning frameworks to optimize its predictive capabilities


- There are two directories named cancer and non-cancer. Each will have directories as patient and vcf files are inside patient directories

- We have GRCh37 as reference

- Forst 5 columns are named as "CHROM"	"POS"	"ID"	"REF"	"ALT" in all vcf/gvcf files

-  A patient may have complete data in single or multiple vcf files but in same patient's directory and directory name is taken as patient name

In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from tabpfn import TabPFNClassifier
from tabpfn.scripts.decision_boundary import DecisionBoundaryDisplay
from sklearn.neighbors import KNeighborsClassifier
import csv
import glob
import pandas as pd
import numpy as np
import hail as hl
# Initialize Hail
hl.init()

Loading BokehJS ...

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/home/mod/env/gene-tool/lib/python3.10/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.3
SparkUI available at http://192.168.1.24:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.125-6e6f46797aed
LOGGING: writing to /home/mod/gen-toolbox/hail-20240222-1741-0.2.125-6e6f46797aed.log


# Data loading and preprocessing

In [2]:
# Initialize Hail's reference genome for GRCh37
rg = hl.get_reference('GRCh37') 

# Add sequence data to the reference genome. Paths are specified in the gen-toolbox configuration.
rg.add_sequence(
    './mod/.vep/homo_sapiens_merged/110_GRCh37/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa',
    './mod/.vep/homo_sapiens_merged/110_GRCh37/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa.fai'
)

# Define the base directory where the data is located
base_dir = '/mnt/sdb/tsch_data

In [3]:
def get_sequence_context(row):
    """
    Retrieves the sequence context around a specified genomic position.
    
    Args:
    row (dict): A dictionary or row containing 'CHROM' and 'POS' keys, representing the chromosome
                and position of interest.
                
    Returns:
    str: A string representing the sequence context around the given position, including 6 bases
         before and 6 bases after the specified position.
    """
    # Extract chromosome and position from the row
    chromosome = row['CHROM']
    position = row['POS']
    
    # Create a Hail locus object for the specified chromosome and position
    locus = hl.locus(chromosome, position)
    
    # Retrieve the sequence context around the locus
    # Note: The original comment mentioned "10 bases before and after" which seems to be a mistake.
    # Corrected to accurately describe the parameters used (6 bases before and after).
    sequence = hl.eval(locus.sequence_context(before=6, after=6))
    
    return sequence

In [ ]:
# Initialize an empty DataFrame for storing Minor Allele Frequency (MAF) data or related information
maf = pd.DataFrame()

In [5]:
paths_list = glob.glob(f'{base_dir}/cancer/*/*') + glob.glob(f'{base_dir}/non-cancer/*/*')

In [6]:
paths_list

['cancer/b/in_silico_sorted2.vcf',
 'cancer/b/in_silico_sorted.vcf',
 'cancer/c/s2.vcf',
 'cancer/c/in_silico_sorted2.vcf',
 'cancer/a/in_silico_sorted.vcf',
 'non-cancer/e/gatk_allsites2.g.vcf',
 'non-cancer/d/gatk_allsites.g.vcf']

In [7]:
def read_df(path):
    """
    Reads a TSV file from the specified path into a pandas DataFrame.
    
    Args:
        path (str): The file path of the TSV file to be read.
        
    Returns:
        pd.DataFrame: A DataFrame containing the data from the TSV file, with the first row of data used as column headers.
    """
    # Use the provided path to open the TSV file
    with open(path, 'r', newline='', encoding='utf-8') as tsvfile:
        # Create a CSV reader object specifying tab ('\t') as delimiter
        reader = csv.reader(tsvfile, delimiter='\t')
        
        # Filter out header lines that start with '##' and load the rest into a DataFrame
        data = [row for row in reader if not row[0].startswith('##')]
        df = pd.DataFrame(data)
    
    # The first row of actual data is used as column headers
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)
    
    # Rename the '#CHROM' column to 'CHROM' for consistency
    df.rename(columns={'#CHROM': 'CHROM'}, inplace=True)
    
    # Optional: Print a confirmation message
    print(f"{path} has been read successfully.")
    
    return df

In [ ]:
for path in paths_list:
    print(f'Reading {path}')
    # Use the refactored read_df function to read the file into a DataFrame
    vcf = read_df(path)
    
    # Select the first five columns and clean up the chromosome names
    vcf = vcf.iloc[:, :5]
    vcf['CHROM'] = vcf['CHROM'].str.replace('chr', '')
    
    # Convert position to numeric, handling errors
    vcf['POS'] = pd.to_numeric(vcf['POS'], errors='coerce')
    
    # Apply the get_sequence_context function to each row
    vcf['Genome_Plus_Minus_10_Bp'] = vcf.apply(get_sequence_context, axis=1)
    
    # Add additional metadata columns
    vcf['Variant_Type'] = 'SNP'
    vcf['Mutation_Status'] = 'Somatic'
    vcf['Matched_Norm_Sample_Barcode'] = '/'.join(path.split('/')[-3:-1])
    
    # Append the processed DataFrame to the master DataFrame
    maf = pd.concat([maf, vcf], ignore_index=True)

reading cancer/b/in_silico_sorted2.vcf
cancer/b/in_silico_sorted2.vcf  has been read
reading cancer/b/in_silico_sorted.vcf
cancer/b/in_silico_sorted.vcf  has been read
reading cancer/c/s2.vcf
cancer/c/s2.vcf  has been read
reading cancer/c/in_silico_sorted2.vcf
cancer/c/in_silico_sorted2.vcf  has been read
reading cancer/a/in_silico_sorted.vcf
cancer/a/in_silico_sorted.vcf  has been read


In [8]:
maf.reset_index(drop=True, inplace=True)
maf

,CHROM,POS,ID,REF,ALT,Genome_Plus_Minus_10_Bp,Variant_Type,Mutation_Status,Matched_Norm_Sample_Barcode
0,1,10318652,rs12402052,C,G,TTTTGCCTATGGG,SNP,Somatic,cancer/b
1,1,10318652,rs12402052,C,G,TTTTGCCTATGGG,SNP,Somatic,cancer/b
2,1,10327623,rs139613776,T,TA,GTATGGTAGGAAA,SNP,Somatic,cancer/b
3,1,10328338,rs1339458,C,T,TTCAGTCTCTAGG,SNP,Somatic,cancer/b
4,1,10328338,rs1339458,C,T,TTCAGTCTCTAGG,SNP,Somatic,cancer/b
...,...,...,...,...,...,...,...,...,...
22142,1,19939,.,T,.,TCTGCATTCGAGG,SNP,Somatic,non-cancer/d
22143,1,19940,.,T,.,CTGCATTCGAGGT,SNP,Somatic,non-cancer/d
22144,1,19941,.,C,.,TGCATTCGAGGTC,SNP,Somatic,non-cancer/d
22145,1,19942,.,G,.,GCATTCGAGGTCC,SNP,Somatic,non-cancer/d


In [10]:
# Extract specific columns from the maf DataFrame
gene_13 = maf['Genome_Plus_Minus_10_Bp']
gene_re = maf['REF']
m_type = maf['Variant_Type']
sta = maf['Mutation_Status']
alle = maf['ALT']
gene_id = maf['Matched_Norm_Sample_Barcode']

# Get unique sample barcodes
unique_ids = gene_id.unique()

# Lists for standardizing mutation notation
sig1_orig = ["G>A", "G>T", "G>C", "A>G", "A>T", "A>C"]
sig1_repl = ["C>T", "C>A", "C>G", "T>C", "T>A", "T>G"]

In [11]:
# Count subscripts that satisfy the conditions
count = 0  # Initialize count to track the number of valid variants
index = []  # Initialize a list to keep track of valid variant indices

# Loop through each variant based on its index
for i in range(len(gene_13)):
    # Check if the variant meets the specified conditions:
    # 1. Variant type (m_type) should be "SNP".
    # 2. Mutation status (sta) should be "Somatic".
    # 3. The sample identifier (gene_id) should be present in the list of unique_ids.
    if m_type[i] == "SNP" and sta[i] == "Somatic" and gene_id[i] in unique_ids:
        # If conditions are met, increment the count and add the index to the list
        count += 1
        index.append(i)

# Initialize variables to track errors
error_num = 0  # Count of variants that fail the sequence context check
error_index = []  # Indices of variants that fail the sequence context check

# Iterate over the indices of variants that initially met the conditions
for i in index:
    # Check if the 7th character of the sequence context (gene_13)
    # does NOT match the reference allele (gene_re).
    # Note: Python uses 0-based indexing, hence [6] accesses the 7th character.
    if gene_13[i][6] != gene_re[i]:
        # If it doesn't match, increment error count and record the index
        error_num += 1
        error_index.append(i)

# Filter out indices of variants that failed the sequence context check
index = [x for x in index if x not in error_index]
# Adjust the count to only include variants that passed all checks
count -= error_num

In [12]:
# Building a one-dimensional feature matrix - initialize mutation type lists
mut1 = []

# One-dimensional feature matrix construction
for i in range(count):
    mutation = gene_13[index[i]][6] + ">" + alle[index[i]]
    mut1.append(mutation)

# Standardize the mutations in the one-dimensional feature matrix
for j in range(len(mut1)):
    for i in range(len(sig1_orig)):
        if mut1[j] == sig1_orig[i]:
            mut1[j] = sig1_repl[i]

# Two-dimensional feature matrices construction with left and right genomic context
mut2_l = []
mut2_r = []
for i in range(count):
    mut2_l.append(gene_13[index[i]][5] + "." + gene_13[index[i]][6] + ">" + alle[index[i]])
    mut2_r.append(gene_13[index[i]][6] + ">" + alle[index[i]] + "." + gene_13[index[i]][7])

# Deduplicate to get unique feature labels
sig2_repl_l = list(set(mut2_l))
sig2_repl_r = list(set(mut2_r))

# Three-dimensional feature matrices construction for left, both sides, and right context
mut3_l = []
mut3_m = []
mut3_r = []
for i in range(count):
    mut3_l.append(gene_13[index[i]][4] + "." + gene_13[index[i]][5] + "." + gene_13[index[i]][6] + ">" + alle[index[i]])
    mut3_m.append(gene_13[index[i]][5] + "." + gene_13[index[i]][6] + ">" + alle[index[i]] + "." + gene_13[index[i]][7])
    mut3_r.append(gene_13[index[i]][6] + ">" + alle[index[i]] + "." + gene_13[index[i]][7] + "." + gene_13[index[i]][8])

# Deduplicate to get unique feature labels
sig3_repl_l = list(set(mut3_l))
sig3_repl_m = list(set(mut3_m))
sig3_repl_r = list(set(mut3_r))

# Combine all unique feature labels for DataFrame columns
feature_columns = sig1_repl + sig2_repl_l + sig2_repl_r + sig3_repl_l + sig3_repl_m + sig3_repl_r

# Initialize a DataFrame with zeros, indexed by unique IDs and columns as feature labels
df_combined = pd.DataFrame(0, index=unique_ids, columns=feature_columns)

# Populate the DataFrame with counts for each mutation type
for i in range(count):
    try:
        df_combined.loc[gene_id[index[i]], mut1[i]] += 1
    except KeyError:
        pass  # Skip if mutation type is not a column in the DataFrame
    try:
        df_combined.loc[gene_id[index[i]], mut2_l[i]] += 1
    except KeyError:
        pass
    try:
        df_combined.loc[gene_id[index[i]], mut2_r[i]] += 1
    except KeyError:
        pass
    try:
        df_combined.loc[gene_id[index[i]], mut3_l[i]] += 1
    except KeyError:
        pass
    try:
        df_combined.loc[gene_id[index[i]], mut3_m[i]] += 1
    except KeyError:
        pass
    try:
        df_combined.loc[gene_id[index[i]], mut3_r[i]] += 1
    except KeyError:
        pass

In [13]:
df_combined

,C>T,C>A,C>G,T>C,T>A,T>G,A.T>A,T.A>C,G.T>C,C.G>C,...,A.A>C.C,T.G>..A,C.A>G.C,T.G>..T,T.C>CT.T,C.T>C.A,G.T>G.G,T.C>T.T,A.A>..C,C.G>C.T
cancer/b,373,78,69,376,36,48,1,8,35,14,...,0,0,15,0,1,2,5,11,0,1
cancer/c,373,78,69,376,36,48,1,8,35,14,...,0,0,15,0,1,2,5,11,0,1
cancer/a,4,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
non-cancer/e,10,1,8,6,5,6,0,0,0,2,...,3,127,1,161,0,2,0,1,146,0
non-cancer/d,10,1,8,6,5,6,0,0,0,2,...,3,127,1,161,0,2,0,1,146,0


In [14]:
# Reset the DataFrame index to convert the index into a column
df_combined.reset_index(inplace=True)

# Rename the newly created column from 'index' to 'label'
# This is useful for further processing or for keeping track of the original index
df_combined.rename(columns={'index': 'label'}, inplace=True)

In [15]:
# Update the 'label' column to binary values based on the presence of 'non-cancer'
# in the original index names. This is typically used for binary classification tasks.
df_combined['label'] = df_combined['label'].apply(lambda x: 0 if 'non-cancer' in x else 1)

# Here's what's happening in the lambda function:
# - If 'non-cancer' is found in the 'label' value, it assigns 0 (indicating non-cancer cases).
# - Otherwise, it assigns 1 (indicating cancer cases or any other case not explicitly labeled as non-cancer).

In [16]:
df_combined

,label,C>T,C>A,C>G,T>C,T>A,T>G,A.T>A,T.A>C,G.T>C,...,A.A>C.C,T.G>..A,C.A>G.C,T.G>..T,T.C>CT.T,C.T>C.A,G.T>G.G,T.C>T.T,A.A>..C,C.G>C.T
0,1,373,78,69,376,36,48,1,8,35,...,0,0,15,0,1,2,5,11,0,1
1,1,373,78,69,376,36,48,1,8,35,...,0,0,15,0,1,2,5,11,0,1
2,1,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,10,1,8,6,5,6,0,0,0,...,3,127,1,161,0,2,0,1,146,0
4,0,10,1,8,6,5,6,0,0,0,...,3,127,1,161,0,2,0,1,146,0


# Splitting the dataset into training and testing sets

In [18]:
X = df_combined.drop('label', axis=1)  # Features
y = df_combined['label']  # Target variable

## Perform the split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### XGBoost Classifier Setup
### Define parameters for XGBoost

##### Adjust depth, loss function and other parameters below

In [ ]:
params = {
    'max_depth': 3,  # Maximum depth of a tree
    'objective': 'binary:logistic',  # Objective function (multiclass classification)
    'eta': 0.3,  # Learning rate 
    'eval_metric': 'logloss', # Evaluation metric
    'random_state': 42 # Seed for reproducibility
}
num_rounds = 100

In [20]:
# Create DMatrix for XGBoost
# DMatrix is an internal data structure that XGBoost uses optimized for both memory efficiency and training speed.
# X_train: Features for the training data.
# y_train: Labels for the training data.
dtrain = xgb.DMatrix(X_train, label=y_train)

# Similarly, prepare the test data as DMatrix for prediction.
# X_test: Features for the test data.
dtest = xgb.DMatrix(X_test, label=y_test)

# Train the XGBoost model
# `params`: Dictionary of parameters for XGBoost.
# `dtrain`: Training data in DMatrix format.
# `num_rounds`: Number of boosting rounds.
bst = xgb.train(params, dtrain, num_rounds)

# Predict with the trained model
# `dtest`: Test data in DMatrix format.
# The `predict` method outputs the probability of each instance being positive (in binary classification).
preds = bst.predict(dtest)

# Convert probabilities to binary output for evaluation
# Threshold of 0.5: If the predicted probability is >0.5, classify as 1 (positive); otherwise, 0 (negative).
preds_binary = [1 if x > 0.5 else 0 for x in preds]

# Calculate and print the accuracy of the model
# `y_test`: True labels for the test set.
# `preds_binary`: Predicted binary labels for the test set.
accuracy = accuracy_score(y_test, preds_binary)
print("XGBoost Accuracy:", accuracy)

Accuracy: 0.0


# Gradient Boosting Classifier

In [21]:
# Gradient Boosting Classifier Setup
# Initialize the classifier with specific hyperparameters:
# - n_estimators: Number of boosting stages to be run. Here, it's set to 100.
# - learning_rate: Rate at which the model learns. A lower rate requires more trees but can lead to better models.
# - random_state: Seed used by the random number generator for reproducibility.
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model on the training data.
# X_train: The training input samples.
# y_train: The target values (class labels) as integers or strings.
clf.fit(X_train, y_train)

# Predict the class labels for the test set.
# X_test: The input samples for testing.
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model.
# y_test: The true labels for the test set.
# y_pred: The predicted labels for the test set.
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy to evaluate the performance of the model.
print("Gradient Boosting Classifier Accuracy:", accuracy)

Accuracy: 1.0


# TabPFN
##### Can use it by reducing features

In [23]:
# import time
# classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=4)
# start = time.time()
# classifier.fit(X_train, y_train)
# y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)
# print('Prediction time: ', time.time() - start, 'Accuracy', accuracy_score(y_test, y_eval))

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


ValueError: ('The number of features for this classifier is restricted to ', 100)

# K-Nearest Neighbors (KNN) Classifier

In [24]:
# Initialize the KNN Classifier
# KNN is a simple, instance-based learning algorithm where the class of a sample is determined by the majority class among its k nearest neighbors.
k = 3  # Specifies the number of neighbors to consider for making predictions. Here, we choose 3 neighbors.
knn_classifier = KNeighborsClassifier(n_neighbors=k)  # Initialize the KNN classifier with the specified number of neighbors.

# Training the Classifier
# The classifier is trained (or "fitted") to the training data, which involves storing the feature vectors and class labels of the training samples.
knn_classifier.fit(X_train, y_train)  # Fit the KNN model to the training data.

# Predicting Labels for the Test Set
# After the model is trained, it predicts the class labels for the provided test data based on the k nearest neighbors.
y_pred = knn_classifier.predict(X_test)  # Use the trained KNN model to predict the class labels for the test set.

# Evaluating Classifier Accuracy
# The accuracy of the predictions is evaluated by comparing the predicted labels against the true labels of the test data.
accuracy = accuracy_score(y_test, y_pred)  # Calculate the accuracy score of the KNN classifier.

# Print the accuracy to provide insight into the performance of the model on unseen data.
print("KNN Classifier Accuracy:", accuracy)

Accuracy: 1.0


In [ ]:
hl.stop()